In [1]:
!pip install torch torchvision torch-fidelity einops accelerate

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, utils

import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import math

# Device: if you're using DDP later, modify this to detect GPU local rank
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# Dataset - CIFAR-10, normalized to [-1, 1]
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # from [0,1] → [-1,1]
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=8, pin_memory=True)

# Cosine beta schedule from Improved DDPM (Nichol & Dhariwal 2021)
def cosine_beta_schedule(timesteps, s=0.008):
    steps = timesteps + 1
    x = torch.linspace(0, timesteps, steps)
    alphas_cumprod = torch.cos(((x / timesteps) + s) / (1 + s) * math.pi / 2) ** 2
    alphas_cumprod = alphas_cumprod / alphas_cumprod[0]
    betas = 1 - (alphas_cumprod[1:] / alphas_cumprod[:-1])
    return torch.clip(betas, 1e-5, 0.999)

Device: cuda


In [27]:
import math

# Timestep Embedding (Sinusoidal)
def get_timestep_embedding(timesteps, embedding_dim):
    half_dim = embedding_dim // 2
    emb = math.log(10000) / (half_dim - 1)
    emb = torch.exp(torch.arange(half_dim, device=timesteps.device) * -emb)
    emb = timesteps[:, None] * emb[None, :]
    emb = torch.cat((emb.sin(), emb.cos()), dim=-1)
    return emb

# Self-Attention Block
class SelfAttention(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.norm = nn.GroupNorm(32, channels)
        self.q = nn.Conv2d(channels, channels, 1)
        self.k = nn.Conv2d(channels, channels, 1)
        self.v = nn.Conv2d(channels, channels, 1)
        self.proj_out = nn.Conv2d(channels, channels, 1)

    def forward(self, x):
        B, C, H, W = x.shape
        h = self.norm(x)
        q = self.q(h).reshape(B, C, -1).permute(0, 2, 1)
        k = self.k(h).reshape(B, C, -1)
        v = self.v(h).reshape(B, C, -1).permute(0, 2, 1)
        attn = torch.bmm(q, k) * (C ** -0.5)
        attn = attn.softmax(dim=-1)
        h = torch.bmm(attn, v)
        h = h.permute(0, 2, 1).reshape(B, C, H, W)
        return x + self.proj_out(h)

# ResBlock with Timestep Conditioning
class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, time_emb_dim, dropout=0.1):
        super().__init__()
        self.norm1 = nn.GroupNorm(32, in_channels)
        self.norm2 = nn.GroupNorm(32, out_channels)
        self.dropout = nn.Dropout(dropout)
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, padding=1)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, padding=1)
        self.time_emb_proj = nn.Linear(time_emb_dim, out_channels)
        self.shortcut = nn.Conv2d(in_channels, out_channels, 1) if in_channels != out_channels else nn.Identity()

    def forward(self, x, t_emb):
        h = self.norm1(x)
        h = F.silu(h)
        h = self.conv1(h)
        time_emb = self.time_emb_proj(F.silu(t_emb))[:, :, None, None]
        h = h + time_emb
        h = self.norm2(h)
        h = F.silu(h)
        h = self.dropout(h)
        h = self.conv2(h)
        return h + self.shortcut(x)

# Full UNet
class UNet(nn.Module):
    def __init__(self, in_channels=3, base_channels=128, channel_mults=(1, 2, 2, 2), time_emb_dim=512):
        super().__init__()
        self.time_mlp = nn.Sequential(
            nn.Linear(1, time_emb_dim),
            nn.SiLU(),
            nn.Linear(time_emb_dim, time_emb_dim)
        )

        self.conv_in = nn.Conv2d(in_channels, base_channels, 3, padding=1)
        self.downs = nn.ModuleList()
        now_channels = base_channels
        self.res_channels = []

        # Downsampling
        for mult in channel_mults:
            out_channels = base_channels * mult
            self.downs.append(nn.ModuleList([
                ResBlock(now_channels, out_channels, time_emb_dim),
                ResBlock(out_channels, out_channels, time_emb_dim),
                SelfAttention(out_channels) if mult >= 2 else nn.Identity(),
                nn.Conv2d(out_channels, out_channels, 3, stride=2, padding=1)
            ]))
            now_channels = out_channels
            self.res_channels.append(now_channels)

        # Middle
        self.mid_block1 = ResBlock(now_channels, now_channels, time_emb_dim)
        self.mid_attn = SelfAttention(now_channels)
        self.mid_block2 = ResBlock(now_channels, now_channels, time_emb_dim)

        # Upsampling
        self.ups = nn.ModuleList()
        for mult in reversed(channel_mults):
            out_channels = base_channels * mult
            self.ups.append(nn.ModuleList([
                ResBlock(now_channels + out_channels, out_channels, time_emb_dim),
                ResBlock(out_channels, out_channels, time_emb_dim),
                SelfAttention(out_channels) if mult >= 2 else nn.Identity(),
                nn.ConvTranspose2d(out_channels, out_channels, 4, stride=2, padding=1)
            ]))
            now_channels = out_channels

        self.conv_out = nn.Sequential(
            nn.GroupNorm(32, now_channels),
            nn.SiLU(),
            nn.Conv2d(now_channels, in_channels, 3, padding=1)
        )

    def forward(self, x, t):
        t_emb = self.time_mlp(t.unsqueeze(-1).float())
        x = self.conv_in(x)
        residuals = []

        for res1, res2, attn, down in self.downs:
            x = res1(x, t_emb)
            x = res2(x, t_emb)
            x = attn(x)
            residuals.append(x)
            x = down(x)

        x = self.mid_block1(x, t_emb)
        x = self.mid_attn(x)
        x = self.mid_block2(x, t_emb)

        for res1, res2, attn, up in self.ups:
            res = residuals.pop()
            if res.shape[-2:] != x.shape[-2:]:
                res = F.interpolate(res, size=x.shape[-2:], mode='nearest')
            x = torch.cat([x, res], dim=1)
            x = res1(x, t_emb)
            x = res2(x, t_emb)
            x = attn(x)
            x = up(x)

        return self.conv_out(x)

In [28]:
class EMA:
    def __init__(self, model, decay=0.9999):
        self.model = model
        self.decay = decay
        self.shadow = {}
        self.backup = {}

    def register(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                self.shadow[name] = param.data.clone()

    def update(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                new_average = (1.0 - self.decay) * param.data + self.decay * self.shadow[name]
                self.shadow[name] = new_average.clone()

    def apply_shadow(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                self.backup[name] = param.data.clone()
                param.data = self.shadow[name]

    def restore(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                param.data = self.backup[name]
        self.backup = {}

In [32]:
# Timesteps
timesteps = 1000

# Get cosine schedule betas
betas = cosine_beta_schedule(timesteps=timesteps)

# Precompute important terms
alphas = 1. - betas
alphas_cumprod = torch.cumprod(alphas, dim=0)
alphas_cumprod_prev = F.pad(alphas_cumprod[:-1], (1,0), value=1.0)

sqrt_alphas_cumprod = torch.sqrt(alphas_cumprod).to(device)
sqrt_one_minus_alphas_cumprod = torch.sqrt(1. - alphas_cumprod).to(device)
betas = betas.to(device)

# Helper function to extract terms per timestep t
def get_index_from_list(vals, t, x_shape):
    vals = vals.to(t.device)
    batch_size = t.shape[0]
    out = vals.gather(-1, t).reshape(batch_size, *((1,) * (len(x_shape) - 1)))
    return out

# Forward q_sample: add noise to a clean image x_start
def q_sample(x_start, t, noise=None):
    if noise is None:
        noise = torch.randn_like(x_start)

    sqrt_alphas_cumprod_t = get_index_from_list(sqrt_alphas_cumprod, t, x_start.shape)
    sqrt_one_minus_alphas_cumprod_t = get_index_from_list(sqrt_one_minus_alphas_cumprod, t, x_start.shape)

    return sqrt_alphas_cumprod_t * x_start + sqrt_one_minus_alphas_cumprod_t * noise

# Loss function (simple MSE between predicted noise and true noise)
mse_loss = nn.MSELoss()

In [33]:
# Create model and move to device
model = UNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=2e-4)

# Take one batch
x, _ = next(iter(train_loader))
x = x.to(device)

# Random timestep for each image
t = torch.randint(0, timesteps, (x.size(0),), device=device).long()

# Add noise to images
noise = torch.randn_like(x)
x_noisy = q_sample(x, t, noise)

# Forward pass through model
model.eval()
with torch.no_grad():
    pred_noise = model(x_noisy, t)

# Compute loss
loss = mse_loss(pred_noise, noise)
print(f" Sanity Check — Initial MSE Loss: {loss.item():.4f}")

 Sanity Check — Initial MSE Loss: 1.1096


In [34]:
import time
from torch.cuda.amp import GradScaler, autocast
from torchvision.utils import make_grid, save_image

# Create model, optimizer, EMA
model = UNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=2e-4)
ema = EMA(model, decay=0.9999)
ema.register()

scaler = GradScaler()
epoch_losses = []

# Create folders
os.makedirs("samples_ddpm", exist_ok=True)
os.makedirs("checkpoints", exist_ok=True)

num_epochs = 800
sample_every = 100

for epoch in range(1, num_epochs + 1):
    model.train()
    epoch_loss = 0
    start_time = time.time()

    for x, _ in tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs}"):
        x = x.to(device)
        t = torch.randint(0, timesteps, (x.size(0),), device=device).long()
        noise = torch.randn_like(x)
        x_noisy = q_sample(x, t, noise)

        with autocast():
            pred = model(x_noisy, t)
            loss = mse_loss(pred, noise)

        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        ema.update()
        epoch_loss += loss.item()

    # Epoch timing and loss logging
    duration = time.time() - start_time
    avg_loss = epoch_loss / len(train_loader)
    epoch_losses.append(avg_loss)
    print(f"Epoch {epoch} completed. Avg Loss: {avg_loss:.4f} | Time: {duration:.2f} sec")

    # Sampling every 100 epochs
    if epoch % sample_every == 0 or epoch == 1:
        model.eval()
        ema.apply_shadow()
        with torch.no_grad():
            sample = torch.randn(16, 3, 32, 32, device=device)
            for i in reversed(range(timesteps)):
                t = torch.full((sample.size(0),), i, device=device, dtype=torch.long)
                noise_pred = model(sample, t)
                alpha = get_index_from_list(alphas, t, sample.shape)
                alpha_cumprod = get_index_from_list(sqrt_alphas_cumprod, t, sample.shape)
                beta = get_index_from_list(betas, t, sample.shape)
                sample = (1 / torch.sqrt(alpha)) * (sample - ((1 - alpha) / (torch.sqrt(1 - alpha_cumprod**2))) * noise_pred)
                if i > 0:
                    noise = torch.randn_like(sample)
                    sample += torch.sqrt(beta) * noise
            sample = (sample.clamp(-1, 1) + 1) / 2
            grid = make_grid(sample, nrow=4)
            save_image(grid, f"samples_ddpm/epoch_{epoch}.png")
        ema.restore()

# Save model
torch.save(model.state_dict(), "checkpoints/ddpm_final_model.pth")
print("Training complete. Model saved.")

# Plot Loss
import matplotlib.pyplot as plt
plt.plot(range(1, len(epoch_losses)+1), epoch_losses)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("DDPM Training Loss Curve")
plt.grid()
plt.savefig("loss_curve.png")
plt.show()

/tmp/ipykernel_1557431/3742997877.py:11: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
Epoch 1/800:   0%|          | 0/391 [00:00<?, ?it/s]/tmp/ipykernel_1557431/3742997877.py:32: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 1/800: 100%|██████████| 391/391 [00:23<00:00, 16.36it/s]


Epoch 1 completed. Avg Loss: 0.2448 | Time: 23.90 sec


Epoch 2/800: 100%|██████████| 391/391 [00:23<00:00, 16.29it/s]


Epoch 2 completed. Avg Loss: 0.0975 | Time: 24.00 sec


Epoch 3/800: 100%|██████████| 391/391 [00:24<00:00, 15.92it/s]


...

Epoch 769/800: 100%|██████████| 391/391 [00:22<00:00, 17.33it/s]


Epoch 769 completed. Avg Loss: 0.0520 | Time: 22.57 sec


Epoch 770/800: 100%|██████████| 391/391 [00:22<00:00, 17.45it/s]


Epoch 770 completed. Avg Loss: 0.0511 | Time: 22.40 sec


Epoch 771/800:  67%|██████▋   | 261/391 [00:14<00:07, 18.26it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [35]:
import torch
import torchvision
from tqdm import tqdm
import os
from PIL import Image

# Create folder
os.makedirs("fid_generated", exist_ok=True)

model.eval()
num_samples = 1000
batch_size = 100
img_size = 32
n_saved = 0

with torch.no_grad():
    for _ in tqdm(range(num_samples // batch_size)):
        x = torch.randn(batch_size, 3, img_size, img_size).to(device)
        for i in reversed(range(timesteps)):
            t = torch.full((batch_size,), i, device=device, dtype=torch.long)
            noise_pred = model(x, t)
            alpha = get_index_from_list(alphas, t, x.shape)
            alpha_cumprod = get_index_from_list(sqrt_alphas_cumprod, t, x.shape)
            beta = get_index_from_list(betas, t, x.shape)

            x = (1 / torch.sqrt(alpha)) * (x - ((1 - alpha) / torch.sqrt(1 - alpha_cumprod**2)) * noise_pred)
            if i > 0:
                noise = torch.randn_like(x)
                x += torch.sqrt(beta) * noise

        x = (x.clamp(-1, 1) + 1) / 2  # scale to [0,1]

        # Save as PNGs
        for img in x:
            img_pil = transforms.ToPILImage()(img.cpu())
            img_pil.save(f"fid_generated/{n_saved:04}.png")
            n_saved += 1

100%|██████████| 10/10 [02:11<00:00, 13.17s/it]


In [42]:
os.makedirs('data/cifar10_val', exist_ok=True)
transform = transforms.Compose([transforms.ToTensor()])

valset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
for i in range(5000):
    img, _ = valset[i]
    img = transforms.ToPILImage()(img)
    img.save(f'data/cifar10_val/{i:04}.png')

In [37]:
from torch_fidelity import calculate_metrics

metrics = calculate_metrics(
    input1='fid_generated',              # generated images
    input2='data/cifar10_val',          # CIFAR-10 validation images
    fid=True,
    feature_layer='2048',
    batch_size=64,
    cuda=True
)

print(f"FID Score: {metrics['frechet_inception_distance']:.2f}")

Creating feature extractor "inception-v3-compat" with features ['2048']
Extracting statistics from input 1
Looking for samples non-recursivelty in "fid_generated" with extensions png,jpg,jpeg
Found 1000 samples
/home/anadar2/.local/lib/python3.12/site-packages/torch_fidelity/datasets.py:16: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  img = torch.ByteTensor(torch.ByteStorage.from_buffer(img.tobytes())).view(height, width, 3)
Processing samples                                                           
Extracting statistics from input 2
Looking for samples non-recursivelty in "data/cifar10_val" with extensions png,jpg,jpeg
Found 1000 samples
Processing samples                                                           


FID Score: 666.52


Frechet Inception Distance: 666.5164685468686


In [38]:
model.load_state_dict(torch.load("checkpoints/ddpm_final_model.pth"))
model.eval()

UNet(
  (time_mlp): Sequential(
    (0): Linear(in_features=1, out_features=512, bias=True)
    (1): SiLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
  )
  (conv_in): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (downs): ModuleList(
    (0): ModuleList(
      (0-1): 2 x ResBlock(
        (norm1): GroupNorm(32, 128, eps=1e-05, affine=True)
        (norm2): GroupNorm(32, 128, eps=1e-05, affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (time_emb_proj): Linear(in_features=512, out_features=128, bias=True)
        (shortcut): Identity()
      )
      (2): Identity()
      (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (1): ModuleList(
      (0): ResBlock(
        (norm1): GroupNorm(32, 128, eps=1e-05, affine=True)
        (no

In [39]:
class EMA:
    def __init__(self, model, decay):
        self.shadow = {}
        self.decay = decay
        self.model = model

    def register(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                self.shadow[name] = param.data.clone()

    def apply_shadow(self):
        for name, param in self.model.named_parameters():
            if name in self.shadow:
                param.data.copy_(self.shadow[name])

    def restore(self):
        for name, param in self.model.named_parameters():
            if name in self.shadow:
                param.data.copy_(self.shadow[name])

In [40]:
ema = EMA(model, decay=0.9999)
ema.register()       # use current weights
ema.apply_shadow()   # apply EMA weights

In [43]:
os.makedirs("fid_ema_generated", exist_ok=True)

model.eval()
n_total = 5000
batch_size = 100
img_size = 32
saved = 0

with torch.no_grad():
    for _ in tqdm(range(n_total // batch_size)):
        x = torch.randn(batch_size, 3, img_size, img_size).to(device)
        for i in reversed(range(timesteps)):
            t = torch.full((batch_size,), i, device=device, dtype=torch.long)
            noise_pred = model(x, t)

            alpha = get_index_from_list(alphas, t, x.shape)
            alpha_cumprod = get_index_from_list(sqrt_alphas_cumprod, t, x.shape)
            beta = get_index_from_list(betas, t, x.shape)

            x = (1 / torch.sqrt(alpha)) * (x - ((1 - alpha) / torch.sqrt(1 - alpha_cumprod**2)) * noise_pred)
            if i > 0:
                noise = torch.randn_like(x)
                x += torch.sqrt(beta) * noise

        x = (x.clamp(-1, 1) + 1) / 2  # scale to [0,1]

        for img in x:
            img_pil = transforms.ToPILImage()(img.cpu())
            img_pil.save(f"fid_ema_generated/{saved:04}.png")
            saved += 1

100%|██████████| 50/50 [10:56<00:00, 13.14s/it]


In [44]:
metrics = calculate_metrics(
    input1='fid_ema_generated',
    input2='data/cifar10_val',
    fid=True,
    cuda=True
)
print(f"FID Score (EMA): {metrics['frechet_inception_distance']:.2f}")

Creating feature extractor "inception-v3-compat" with features ['2048']
Extracting statistics from input 1
Looking for samples non-recursivelty in "fid_ema_generated" with extensions png,jpg,jpeg
Found 5000 samples
Processing samples                                                            
Extracting statistics from input 2
Looking for samples non-recursivelty in "data/cifar10_val" with extensions png,jpg,jpeg
Found 5000 samples
Processing samples                                                            


FID Score (EMA): 649.07


Frechet Inception Distance: 649.0736533887051


In [45]:
import shutil
import zipfile
import os

# Output zip filename
zip_filename = 'ddpm_outputs.zip'

# Create the zip file
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Add checkpoints folder
    for root, dirs, files in os.walk('checkpoints'):
        for file in files:
            file_path = os.path.join(root, file)
            zipf.write(file_path, arcname=os.path.relpath(file_path, start='.'))

    # Add fid_ema_generated folder
    for root, dirs, files in os.walk('fid_ema_generated'):
        for file in files:
            file_path = os.path.join(root, file)
            zipf.write(file_path, arcname=os.path.relpath(file_path, start='.'))

print(f"Created zip: {zip_filename}")

TypeError: 'str' object does not support the context manager protocol